In [13]:
# coding: utf-8
"""
gensimを用いたLDA
purityの計算
"""

from gensim import corpora, models, similarities
import csv
import MeCab
import pickle
import collections
import numpy as np

# csvファイルの読み込み
def readcsv(path):
    f = open(path, "rb")
    dataReader = csv.reader(f)
    arr = [row for row in dataReader]
    return arr

def writecsv(arr, path):
    f = open(path, "ab")
    dataWriter = csv.writer(f)
    dataWriter.writerows(arr)
    f.close()

def readdump(path):
    f = open(path, "r")
    arr = pickle.load(f)
    f.close()
    return arr

# f_measureを計算する
def cal_f_measure(list_predict_measure):
    # 生成したクラスタ内のカウント
    dict_predict_cluster = collections.defaultdict(list)
    for row in list_predict_measure:
        dict_predict_cluster[row[0]].append(row[1])
        
    # もとあるクラス内のカウント
    dict_measure_cluster = collections.defaultdict(list)
    for row in list_predict_measure:
        dict_measure_cluster[row[1]].append(row[0])
    
    # local_purityの計算
    list_purity = []
    for row in dict_predict_cluster.items():
        major_class = sorted(collections.Counter(row[1]).items(), key=lambda x: x[1], reverse=True)[0][1]
        class_num = len(row[1])
        list_purity.append([major_class, class_num])
    purity = float(np.sum(zip(*list_purity)[0])) / np.sum(zip(*list_purity)[1])
    print "Purity: ", purity
    
    # inverse_purityの計算
    list_inverse_purity = []
    for row in dict_measure_cluster.items():
        major_class = sorted(collections.Counter(row[1]).items(), key=lambda x: x[1], reverse=True)[0][1]
        class_num = len(row[1])
        list_inverse_purity.append([major_class, class_num])
    inverse_purity = float(np.sum(zip(*list_inverse_purity)[0])) / np.sum(zip(*list_inverse_purity)[1])
    print "Inverse Purity: ", inverse_purity
    
    print "F-value: ", 2 / (1 / purity + 1 / inverse_purity)

### コーパスの読み込み
* 1行ずつ形態素解析して、ラベルが付いたものと、口コミごとに形態素解析しているのものの2種類を読み込む

In [14]:
list_sep_words_class = readdump("./files/list_sep_words_label.dump")
list_bag_of_words, list_class = zip(*list_sep_words_class)
list_bag_of_words_1 = list(list_bag_of_words)
list_class = list(list_class)

list_bag_of_words_2 = readdump("./files/list_sep_words_per_human.dump")

### LDAにより、トピックの抽出
1. 辞書の作成
2. 辞書を用いてコーパスのマッピングをする
3. マッピングしたコーパスを元にLDA
4. LDAの結果を表示

In [15]:
# 辞書の作成
dictionary = corpora.Dictionary(list_bag_of_words_2)
# コーパスをマッピングし、LDA
corpus_1 = [dictionary.doc2bow(text) for text in list_bag_of_words_1]
corpus_2 = [dictionary.doc2bow(text) for text in list_bag_of_words_2]
lda = models.LdaModel(corpus=corpus_2, id2word=dictionary, num_topics=7)
# p(word|topic)を表示
for i, row in enumerate(lda.show_topics(0)):
    print row

0.019*サービス + 0.010*設備 + 0.010*雰囲気 + 0.008*非常 + 0.008*生活 + 0.008*必要 + 0.007*高い + 0.006*常駐 + 0.006*印象 + 0.006*病院
0.011*病院 + 0.009*サービス + 0.009*ホテル + 0.009*高級 + 0.008*感じ + 0.008*充実 + 0.008*高い + 0.007*利用 + 0.007*雰囲気 + 0.006*車
0.009*高い + 0.008*場所 + 0.008*人 + 0.008*メニュー + 0.008*職員 + 0.007*明るい + 0.007*利用 + 0.007*印象 + 0.007*病院 + 0.007*いい
0.012*メニュー + 0.008*大変 + 0.007*印象 + 0.007*充実 + 0.007*祖母 + 0.007*広い + 0.007*やすい + 0.006*バス + 0.006*感じ + 0.006*方々
0.012*便利 + 0.011*ホテル + 0.010*高級 + 0.009*駅 + 0.008*高い + 0.007*充実 + 0.007*病院 + 0.006*レストラン + 0.006*居室 + 0.006*価格
0.011*感じ + 0.008*駅 + 0.008*雰囲気 + 0.007*バス + 0.007*印象 + 0.007*メニュー + 0.007*サービス + 0.007*見学 + 0.007*やすい + 0.007*ホテル
0.013*人 + 0.010*明るい + 0.009*感じ + 0.008*必要 + 0.007*高級 + 0.007*サービス + 0.007*駅 + 0.006*レストラン + 0.006*自分 + 0.006*参加


### 文の予測
1. LDAの結果から各文のクラスを予測
2. 予測ラベルと結果ラベルのリストを作成

In [16]:
list_predict_measure = []
error_count = 0
for i in range(len(corpus_1)):
    predict_class = sorted(lda[corpus_1[i]], key=lambda x:x[1], reverse=True)[0][0]
    measure_class = list_class[i]
    try:
        list_predict_measure.append([predict_class, int(measure_class)])
    except:
        error_count += 1
print "エラーしたコーパス", error_count

エラーしたコーパス 7


In [17]:
list_predict, list_class = zip(*list_predict_measure)
collections.Counter(list(list_predict))

Counter({1: 963, 6: 596, 2: 457, 4: 407, 0: 355, 3: 311, 5: 265})

### 定量評価 (purity)

In [18]:
cal_f_measure(list_predict_measure)

Purity:  0.243291592129
Inverse Purity:  0.287119856887
F-value:  0.263394944598
